In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
#model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelWithLMHead.from_pretrained("/content/drive/MyDrive/gpt/sc_pairs/checkpoint")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1595: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(device)

cpu


In [ ]:
import pandas as pd
import numpy as np
import torch
import json
import glob
import logging
import os
import pickle
import random
import re
import shutil

In [ ]:
from typing import Dict, List, Tuple
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange
from pathlib import Path
from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)
try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [ ]:
class Args():
    def __init__(self):
        self.output_dir = '/content/drive/MyDrive/katz_2024/train'
        self.model_type = 'gpt2'
        self.model_name_or_path = '/content/drive/MyDrive/gpt/sc_pairs/checkpoint'
        #self.model_name_or_path ='microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 150
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 10000
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

In [ ]:
import json

f_train = open("/content/drive/MyDrive/gpt/data/data/train_data.json")
train_data = json.load(f_train)
f_train.close()
# print(len(train_data))



In [ ]:
f_validate = open("/content/drive/MyDrive/gpt/data/data/val_data.json")
validate_data = json.load(f_validate)
f_validate.close()
# print(len(validate_data))

In [ ]:
train_contexted = []

for row in train_data:
    row_data = [row['answer'], row['question']]
    train_contexted.append(row_data)


In [ ]:
validate_contexted = []

for row in validate_data:
    row_data = [row['answer'], row['question']]
    validate_contexted.append(row_data)

In [ ]:
columns = ['response', 'context']
columns = columns + ['context/'+str(i) for i in range(0)]
columns

['response', 'context']

In [ ]:
len(train_contexted)
trn_df = pd.DataFrame.from_records(train_contexted, columns=columns)
trn_df.head(5)

,response,context
0,"The prerequisites for ECO 1101, Intermediate M...","What are the prerequisites for ECO 1101, Inter..."
1,Gold was honored to light the torch during the...,Who had the honor of lighting the torch during...
2,Daniel Davidson is an instructor in the Katz S...,Who is Daniel Davidson?
3,"Esteban Wesson, an industry professor, teaches...",What expertise does Esteban Wesson bring to Ye...
4,You can apply for oncampus housing for the 202...,How can one apply for oncampus housing at Yesh...


In [ ]:
len(validate_contexted)
val_df = pd.DataFrame.from_records(validate_contexted, columns=columns)
val_df.head(5)

,response,context
0,Dr. Barrera has been featured in Womens Health...,What publications has Dr. Barrera been feature...
1,Gabriel Cwilichs secondary office is located i...,What is Gabriel Cwilichs secondary office loca...
2,Samuel Strum is the director of data technolog...,What is Samuel Strums current position?
3,He is an assistant professor in the Department...,Where does Dr. Ran Drori work?
4,Yeshiva Universitys Wilf Campus in Washington ...,What is the significance of Yeshiva University...


In [ ]:
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [ ]:
# Cacheing and storing of data/checkpoints

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

In [ ]:
trn_df

,response,context
0,"The prerequisites for ECO 1101, Intermediate M...","What are the prerequisites for ECO 1101, Inter..."
1,Gold was honored to light the torch during the...,Who had the honor of lighting the torch during...
2,Daniel Davidson is an instructor in the Katz S...,Who is Daniel Davidson?
3,"Esteban Wesson, an industry professor, teaches...",What expertise does Esteban Wesson bring to Ye...
4,You can apply for oncampus housing for the 202...,How can one apply for oncampus housing at Yesh...
...,...,...
5863,"To live better with illness, injury, and disab...","What can be done to live better with illness, ..."
5864,The phone number for AOTA is 3016526611.,What is the phone number for AOTA?
5865,Students have gained invaluable industry exper...,What kind of industry experience have students...
5866,WeberLopez practices primary careurgent care m...,What type of medicine does Joseph WeberLopez p...


In [ ]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError(" install apex from  to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of  model
def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info(" Running evaluation {} ".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [ ]:
def main(df_trn, df_val):
    args = Args()

    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError(" no checkpoint was found.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelWithLMHead.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)

    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer,reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [ ]:
main(trn_df, val_df)

In [ ]:
f_test = open("/content/drive/MyDrive/katz_2024/Test_QA_pairs.json")
test_data = json.load(f_test)
test_query = []
test_response = []

for entry in test_data:
    test_response.append(entry["Answer"])
    test_query.append(entry["Question"])

print(len(test_response))
print(len(test_query))

2081
2081


In [ ]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Your code here


In [ ]:
import pandas as pd
import warnings


# Your existing code to run the chatbot
test_chatbot = []

for i in range(len(test_query)):
    #tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small', padding_side='left')
    #tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small', padding=True, max_length=100, truncation=True)
    tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small', padding=True, max_length=100, truncation=True, padding_side='right')
    model = AutoModelWithLMHead.from_pretrained('/content/drive/MyDrive/katz_2024/train/checkpoint-70000')
    # append the new user input tokens to the chat history
    bot_input_ids = tokenizer.encode(test_query[i] + tokenizer.eos_token, return_tensors='pt')
    print("UserInput: {} \n".format(test_query[i]))
    print("Reference:  {} \n".format(test_response[i]))

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(
        bot_input_ids, max_length=100,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=10,
        top_p=0.7,
        temperature=0.8
    )

    # pretty print last ouput tokens from bot
    print("Predict: {} \n\n".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
    chatbot_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    test_chatbot.append(chatbot_response)

print(len(test_chatbot))

# Create a list of dictionaries
data = []
for i in range(len(test_query)):
    data.append({
        "UserInput": test_query[i],
        "Reference": test_response[i],
        "Predict": test_chatbot[i]
    })

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/katz_2024/test_data.csv', index=False)

print("Chatbot responses saved to chatbot_output.csv")


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


UserInput: What is the student/faculty ratio at this university? 

Reference:  The student/faculty ratio at Yeshiva University is 7:1. 

Predict: The student or faculty member working in the faculty at this school is 95 fulltime, with some positions requiring a minimum grade of Bplus. 




A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


UserInput: How often do students get to interact with professors outside of class? 

Reference:  Students at Yeshiva University have ample opportunities to interact with professors outside of class through office hours, research projects, and mentoring programs. 

Predict: Students often get to chat with faculty during class. 




A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

UserInput: How did Zesarae Bodie contribute during the COVID-19 pandemic? 

Reference:  During the COVID-19 pandemic, Zesarae Bodie worked as a frontline worker, assisting in rehabilitating the elderly and continuing to provide in-home services throughout New York City. 

Predict: During the COVIS19 pandemotional year, Zesar began his work as a frontline worker, assisting in rehabilitating the elderly and continuing to provide inhome services throughout New York City. 




A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


UserInput: What are Zesarae Bodie's personal thoughts on joining Yeshiva University? 

Reference:  Zesarae Bodie expressed excitement and gratitude for the opportunity to merge her passions for education and occupational therapy at Yeshiva University, looking forward to sharing her knowledge and experiences with future therapists. 

Predict: Zesaraele Bodie expressed excitement and gratitude for the opportunity to merge her passions for education and occupational therapy at Yeshivas new school, looking forward to sharing her knowledge and experiences with future therapists. 




A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


UserInput: How can someone contact Zesarae Bodie? 

Reference:  Zesarae Bodie can be contacted via email at zesarae.bodie@yu.edu, by phone at 646-592-4730, or through her office at the Dora Rousso Building, Room 405, Jack and Pearl Resnick Campus, 1165 Morris Park Ave, Bronx, NY 10461. 

Predict: Zesaraae Bodies personal website is zesaraebodie.bodie@yu.edu, and she is available to answer questions about academic programs, admission, and campus life. 


2081
Chatbot responses saved to chatbot_output.csv


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

def generate_response(model, tokenizer, prompt):
    # Encode user input and generate chatbot response
    bot_input_ids = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors='pt', max_length=100, truncation=True)
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=100,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=10,
        top_p=0.7,
        temperature=0.8
    )

    # Decode and return chatbot response
    chatbot_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return chatbot_response

# Load the pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelWithLMHead.from_pretrained('/content/drive/MyDrive/gpt-final1/checkpoint-90000')

# Test the chatbot interactively
while True:
    # Take user input
    user_input = input("UserInput: ")

    # Check for exit command
    if user_input.lower() == 'exit':
        print("Exiting the chatbot.")
        break

    # Generate and print chatbot response
    chatbot_response = generate_response(model, tokenizer, user_input)
    print("Generated response:", chatbot_response)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1595: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


OSError: Incorrect path_or_model_id: '/content/drive/MyDrive/gpt-final1/checkpoint-90000'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead

def generate_response(model, tokenizer, prompt):
    # Encode user input and generate chatbot response
    bot_input_ids = tokenizer.encode(prompt + tokenizer.eos_token, return_tensors='pt')
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=100,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=10,
        top_p=0.7,
        temperature=0.8
    )

    # Decode and return chatbot response
    chatbot_response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return chatbot_response

# Load the pre-trained tokenizer and model with padding_side='left'
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small', padding_side='left')
model = AutoModelWithLMHead.from_pretrained('/content/drive/MyDrive/gpt-final1/checkpoint-90000')

# Test the chatbot interactively
while True:
    # Take user input
    user_input = input("UserInput: ")

    # Check for exit command
    if user_input.lower() == 'exit':
        print("Exiting the chatbot.")
        break

    # Generate and print chatbot response
    chatbot_response = generate_response(model, tokenizer, user_input)
    print("Generated response:", chatbot_response)


UserInput: hi


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Generated response: Barrera serves as the Assistant Director of Student Aid at Yeshiva University, and you can contact her at bosum.garra@yu.edu


KeyboardInterrupt: Interrupted by user

In [ ]:
# Test the chatbot
prompt = "What is MS AI program in Yeshiva University?"
response = generate_response(model, tokenizer, prompt)
print(prompt)
print("Generated response:", response)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


What is MS AI program in Yeshiva University?
Generated response: MS AI program is an AI and machine learning technique that focuses on enabling computers to interpret and understand visual information from the world around them.


In [ ]:
with open('gpt2-results.txt', 'w') as f:
    for i in test_chatbot:
        f.write('Chatbot: %s\n\n' % i)

In [ ]:
with open('gpt2-results-conversation.txt', 'w') as f:
    for i in range(len(test_chatbot)):
        f.write("--------------------example %s--------------------\n" %(i+1))
        f.write('Patient: %s\n' % test_query[i])
        f.write('Reference: %s\n' % test_response[i])
        f.write('Predict: %s\n' % test_chatbot[i])
        f.write('\n')

In [ ]:
#!pip install "nltk==3.4.5"

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.translate.nist_score import sentence_nist


In [ ]:
def get_metrics(pred, target):
    turns = len(target)
    bleu_2 = 0
    bleu_4 = 0
    meteor = 0
    nist_2 = 0
    nist_4 = 0
    for index in range(turns):
        pred_utt = pred[index]
        target_utt = target[index]
        min_len = min(len(pred_utt), len(target_utt))
        lens = min(min_len, 4)
        if lens == 0:
            continue
        if lens >= 4:
            bleu_4_utt = sentence_bleu([target_utt], pred_utt, weights = (0.25, 0.25, 0.25, 0.25), smoothing_function = SmoothingFunction().method1)
            nist_4_utt = sentence_nist([target_utt], pred_utt, 4)
        else:
            bleu_4_utt = 0
            nist_4_utt = 0
        if lens >= 2:
            bleu_2_utt = sentence_bleu([target_utt], pred_utt, weights = (0.5, 0.5), smoothing_function = SmoothingFunction().method1)
            nist_2_utt = sentence_nist([target_utt], pred_utt, 2)
        else:
            bleu_2_utt = 0
            nist_2_utt = 0

        bleu_2 += bleu_2_utt
        bleu_4 += bleu_4_utt
        meteor += meteor_score([" ".join(target_utt)], " ".join(pred_utt))
        nist_2 += nist_2_utt
        nist_4 += nist_4_utt

    bleu_2 /= turns
    bleu_4 /= turns
    meteor /= turns
    nist_2 /= turns
    nist_4 /= turns
    return bleu_2, bleu_4, meteor, nist_2, nist_4

In [ ]:
mkdir nltk_data

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.translate import meteor_score

# Example sentences (replace these with your actual sentences)
hypothesis = "Your untokenized hypothesis sentence here."
reference = "Your untokenized reference sentence here."

# Tokenize your hypothesis and reference sentences
hypothesis_tokens = word_tokenize(hypothesis)
reference_tokens = word_tokenize(reference)

# Calculate METEOR score
score = meteor_score.meteor_score([reference_tokens], hypothesis_tokens)

# Print the METEOR score
print("METEOR Score:", score)


METEOR Score: 0.8066666666666668


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.nist_score import sentence_nist

# Example sentences (replace these with your actual sentences)
test_chatbot = ["Your test chatbot response 1", "Your test chatbot response 2"]
test_response = "Your test response"

# Tokenize the test chatbot responses
test_chatbot_tokens = [nltk.word_tokenize(response) for response in test_chatbot]
test_response_tokens = nltk.word_tokenize(test_response)

# Calculate BLEU-2 scores for each chatbot response
bleu_2_scores = [sentence_bleu([test_response_tokens], response, weights=(0.5, 0.5), smoothing_function=SmoothingFunction().method7) for response in test_chatbot_tokens]

# Calculate BLEU-4 scores for each chatbot response
bleu_4_scores = [sentence_bleu([test_response_tokens], response, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=SmoothingFunction().method7) for response in test_chatbot_tokens]


# Calculate NIST scores for each chatbot response
nist_scores = [sentence_nist([test_response_tokens], response) for response in test_chatbot_tokens]

# Print the BLEU and NIST scores
print("BLEU-2 Scores:", bleu_2_scores)
print("BLEU-4 Scores:", bleu_4_scores)
print("NIST Scores:", nist_scores)


BLEU-2 Scores: [0.5388602512436507, 0.5388602512436507]
BLEU-4 Scores: [0.23603634562950465, 0.23603634562950465]
NIST Scores: [0.9509775004326938, 0.9509775004326938]


In [ ]:
#test_data

In [ ]:
#rouge score

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

hypothesis = " MS AI program is an AI and machine learning technique"

reference = "The phone number for the Steinberg Library is (212) 340-7720."

rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
scores

[{'rouge-1': {'r': 0.1, 'p': 0.1111111111111111, 'f': 0.1052631529085875},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.1, 'p': 0.1111111111111111, 'f': 0.1052631529085875}}]

In [ ]:
preds = pd.read_csv('/content/drive/MyDrive/katz_2024/test_data.csv')

In [ ]:
preds.head()

,UserInput,Reference,Predict
0,What is the student/faculty ratio at this univ...,The student/faculty ratio at Yeshiva Universit...,The student or faculty member working in the f...
1,How often do students get to interact with pro...,Students at Yeshiva University have ample oppo...,Students often get to chat with faculty during...
2,How active is the student community on campus?,The student community at Yeshiva University is...,The student body is expressed in a global Face...
3,Can you tell me more about the extracurricular...,Yeshiva University offers a wide range of extr...,The extracursricular activities at this school...
4,What percentage of students get financial aid?,Approximately 85% of students at Yeshiva Unive...,Approximately threequarters of students receiv...


In [ ]:
rouge_scores = []
act_ans= []
pred_ans= []
for index, row in preds.iterrows():

  act_ans = row['Reference']
  pred_ans = row['Predict']

  rouge = Rouge()
  scores = rouge.get_scores(act_ans, pred_ans,avg=True)
  rouge_scores.append(scores)

In [ ]:
from rouge import Rouge

rouge_scores = []

# Assuming preds is a DataFrame
for index, row in preds.iterrows():
    # Ensure that row['Reference'] and row['Predict'] are strings
    act_ans = str(row['Reference'])
    pred_ans = str(row['Predict'])

    rouge = Rouge()
    scores = rouge.get_scores(act_ans, pred_ans, avg=True)
    rouge_scores.append(scores)


In [ ]:
rouge_scores

[{'rouge-1': {'r': 0.13043478260869565, 'p': 0.375, 'f': 0.1935483832674298},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.13043478260869565, 'p': 0.375, 'f': 0.1935483832674298}},
 {'rouge-1': {'r': 0.4444444444444444,
   'p': 0.18181818181818182,
   'f': 0.25806451200832464},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.4444444444444444,
   'p': 0.18181818181818182,
   'f': 0.25806451200832464}},
 {'rouge-1': {'r': 0.4117647058823529,
   'p': 0.30434782608695654,
   'f': 0.3499999951125},
  'rouge-2': {'r': 0.058823529411764705,
   'p': 0.043478260869565216,
   'f': 0.04999999511250048},
  'rouge-l': {'r': 0.35294117647058826,
   'p': 0.2608695652173913,
   'f': 0.29999999511250003}},
 {'rouge-1': {'r': 0.15789473684210525,
   'p': 0.3157894736842105,
   'f': 0.21052631134502933},
  'rouge-2': {'r': 0.024390243902439025,
   'p': 0.05555555555555555,
   'f': 0.03389830084458543},
  'rouge-l': {'r': 0.15789473684210525,
   'p': 0.31578947368421

In [ ]:
act_ans = []
pred_ans = []

for index, row in preds.iterrows():

  act_ans.append(row['Reference'])
  pred_ans.append(row['Predict'])

In [ ]:
rouge_scores = []
hyps = []  # Assuming hyps is a list of predicted answers
refs = []  # Assuming refs is a list of reference answers

for index, row in preds.iterrows():
    # Ensure that row['Reference'] and row['Predict'] are strings
    act_ans = str(row['Reference'])
    pred_ans = str(row['Predict'])

    hyps.append(pred_ans)
    refs.append(act_ans)

rouge = Rouge()
scores = rouge.get_scores(hyps, refs, avg=True)
rouge_scores.append(scores)
scores

{'rouge-1': {'r': 0.42519932933744475,
  'p': 0.4992061506711907,
  'f': 0.44647341738113144},
 'rouge-2': {'r': 0.31375593161960896,
  'p': 0.3392882598679974,
  'f': 0.3199920258650268},
 'rouge-l': {'r': 0.4083762531573302,
  'p': 0.47338375053039944,
  'f': 0.426824078084975}}

In [ ]:
from tabulate import tabulate

In [ ]:
# Create a list of dictionaries for tabulating the scores
score_table = [{'Metric': metric, 'Precision': score['p'], 'Recall': score['r'], 'F1-Score': score['f']} for metric, score in scores.items()]

# Print the scores in a tabular format
print(tabulate(score_table, headers='keys', tablefmt='grid'))

+----------+-------------+----------+------------+
| Metric   |   Precision |   Recall |   F1-Score |
+==========+=============+==========+============+
| rouge-1  |    0.499206 | 0.425199 |   0.446473 |
+----------+-------------+----------+------------+
| rouge-2  |    0.339288 | 0.313756 |   0.319992 |
+----------+-------------+----------+------------+
| rouge-l  |    0.473384 | 0.408376 |   0.426824 |
+----------+-------------+----------+------------+


In [ ]:
!pip install streamlit


In [ ]:
import streamlit as st

def main():
    st.title("Chat-bot")
    st.write("Hi")

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'streamlit'

In [ ]:
%streamlit run my_app.py
